In [1]:
import sys
sys.path.append("../code/")


In [2]:
import datetime

from sklearn.model_selection import train_test_split, KFold

from common import compute_rmse_fraction, contri, get_tensor, create_region_df_dfc_static
from create_matrix import *
from tensor_custom_core import *

In [3]:
target_df, target_dfc, target_tensor, target_static = create_region_df_dfc_static('Austin', 2014)
target_L = get_L(target_static)


In [4]:
case = 2
tensor = target_tensor
num_home = 3
num_season = 3
iters = 1300
lr = 0.1

In [5]:
%%time
H, A, T, Hs, As, Ts, HATs, costs = learn_HAT_adagrad_graph(case, tensor, target_L, num_home, num_season, num_iter = iters, lr = lr)

CPU times: user 36.3 s, sys: 228 ms, total: 36.5 s
Wall time: 9.14 s


### Try PyTorch

In [6]:
import torch
from torch.autograd import Variable

In [7]:
torch.manual_seed(42)
np.random.seed(42)

In [8]:
M, N,O = tensor.shape

In [10]:
params = {}
params['M'], params['N'], params['O'] = tensor.shape
params['a'] = num_home
params['b'] = num_season
H_dim_chars = list(cases[case]['HA'].split(",")[0].strip())
H_dim = tuple(params[x] for x in H_dim_chars)
A_dim_chars = list(cases[case]['HA'].split(",")[1].split("-")[0].strip())
A_dim = tuple(params[x] for x in A_dim_chars)
S_dim_chars = list(cases[case]['HAT'].split(",")[1].split("-")[0].strip())
S_dim = tuple(params[x] for x in S_dim_chars)


In [12]:
mask = ~np.isnan(tensor)
ones = np.ones(mask.shape)
mask = (mask*ones).astype(int)
mask = Variable(torch.ByteTensor(mask))

In [13]:
%%time
H = Variable(torch.randn(*H_dim), requires_grad=True)
A = Variable(torch.randn(*A_dim), requires_grad=True)
S = Variable(torch.randn(*S_dim), requires_grad=True)
tensor_copy = Variable(torch.Tensor(tensor), requires_grad=False)
tensor_copy[tensor_copy != tensor_copy] = 0

learning_rate = 0.1
optimizer = torch.optim.Adam([H, A, S], lr=learning_rate)

for t in range(1300):
    
    HA = torch.mm(H, A.view(params['a'], params['N']*params['b'])).view(params['M'], params['N'], params['b'])
    HAS = torch.mm(HA.view(params['M']*params['N'], params['b']), S.transpose(0, 1)).view(params['M'], params['N'], params['O'])
        
    loss_all = (HAS - tensor_copy).pow(2)
    loss_masked = Variable(torch.Tensor(M, N, O).zero_()).masked_copy(mask, loss_all)
    loss = torch.sqrt(loss_masked.mean())
        
    if t % 10 == 0:
        print(t, loss.data[0])
    
    optimizer.zero_grad()   
    
    loss.backward()
    
    optimizer.step()

/home/yj9xs/anaconda2/lib/python2.7/site-packages/torch/autograd/variable.py:699: UserWarning: masked_copy is deprecated and renamed to masked_scatter, and will be removed in v0.3
  warnings.warn("masked_copy is deprecated and renamed to masked_scatter, and will be removed in v0.3")


(0, 237.4827117919922)
(10, 234.4619598388672)
(20, 217.34559631347656)
(30, 177.96591186523438)
(40, 123.04818725585938)
(50, 97.9781494140625)
(60, 90.14020538330078)
(70, 86.66566467285156)
(80, 83.679931640625)
(90, 80.75070190429688)
(100, 78.82691192626953)
(110, 77.34779357910156)
(120, 76.09830474853516)
(130, 74.8541030883789)
(140, 73.33822631835938)
(150, 71.34660339355469)
(160, 68.76170349121094)
(170, 66.07138061523438)
(180, 64.0447998046875)
(190, 62.74252700805664)
(200, 62.012855529785156)
(210, 61.569881439208984)
(220, 61.285491943359375)
(230, 61.10401153564453)
(240, 60.98958206176758)
(250, 60.9160041809082)
(260, 60.86655044555664)
(270, 60.83244705200195)
(280, 60.80561447143555)
(290, 60.78407669067383)
(300, 60.7645263671875)
(310, 60.74620056152344)
(320, 60.72829818725586)
(330, 60.71091842651367)
(340, 60.69142150878906)
(350, 60.67185974121094)
(360, 60.6512451171875)
(370, 60.62892150878906)
(380, 60.60439682006836)
(390, 60.5779914855957)
(400, 60.54874